In [ ]:
import pandas as pd
import numpy as np

In [ ]:
catalog = pd.read_excel("Microsoft-Lily Demo Product Catalog.xlsx")
attributes = pd.read_csv("Microsoft-Lily Demo Catalog Attributes.csv")

In [ ]:
catalog.head()

In [ ]:
attributes.head()

In [ ]:
import requests

service_name = ''
api_key = ''

index_name = "lily-ai-v3"

api_version = '2021-04-30-preview'

headers = {
    "api-key": api_key,
    "content-type": "application/json"
}

In [ ]:
index_data_url = 'https://{}.search.windows.net/indexes/{}/docs/index?api-version={}'.format(service_name, index_name, api_version)
documents = { "value": [] }
count = 0

for index, row in catalog.iterrows():
    item = dict(row)
    
    if str(item['bullet_text']) == "nan":
        item.pop('bullet_text')
    else:
        item['bullet_text'] = str(item['bullet_text']).split("|") 
        item['bullet_text'] = [t.strip() for t in item['bullet_text']]
    item['sku_id'] = str(item['sku_id'])
    item['parent_id'] = str(item['parent_id'])
    item['images'] = item['images'].split("|") 
    item['images'] = [t.strip() for t in item['images']]
    item['product_type'] = item['site_product_type'].lower()
    item.pop("site_product_type")
    
    item['image_urls'] = item['images']
    item.pop("images")
    
    if str(item['product_care']) == "nan":
        item.pop("product_care")
        
    if str(item['fabric_content']) == "nan":
        item.pop("fabric_content")
    else:
        item['fabric_content'] = [item['fabric_content']]
    
    #print(item['product_care'])
    
    matching_attributes = attributes[attributes['sku_id'] == row['sku_id']]
    
    for a_index, a_row in matching_attributes.iterrows():
        key = str(a_row['category']).lower().replace(' ', '_')
        if key in item:
            item[key].append(str(a_row['attribute']))
        else:
            item[key] = []
            item[key].append(str(a_row['attribute']))
        
    item["@search.action"] =  "mergeOrUpload"
    
    if "nan" in item:
        item.pop("nan")
    
    #print(item)
    #break
    
    documents["value"].append(item)
    count += 1
    
    if count % 1000 == 0:
        #print(documents)
        #print()
        response = requests.post(index_data_url, json = documents, headers=headers)
        print(response)
        documents = { "value": [] }